<a href="https://colab.research.google.com/github/gourav-prateek-sharma/Capstone-Project-COVER/blob/main/Classification_Task_II(All%20Obstacle%20Config).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Grid Code**

In [3]:
import numpy as np
import itertools
import matplotlib.pyplot as plt
def create_grid(A, M, obstacle_prob=0.1):
    N = int(np.sqrt(M))
    grid = np.zeros((N, N), dtype=int)
    num_obstacles = int(M * obstacle_prob)
    obstacle_positions = np.random.choice(M, num_obstacles, replace=True)

    for pos in obstacle_positions:
        grid[pos // N, pos % N] = 1
    return grid

In [4]:
def plot_grid(grid,tx):
    plt.figure(figsize=(int(np.sqrt(M)), int(np.sqrt(M))))
    plt.imshow(grid, cmap='viridis')

    # Add gridlines
    plt.grid(visible=True, color='white', linewidth=1, which='minor')

    # Setting labels
    plt.xticks(np.arange(grid.shape[1]), labels=np.arange(grid.shape[1]))
    plt.yticks(np.arange(grid.shape[0]), labels=np.arange(grid.shape[0]))

    # Setting gridlines
    plt.xticks(np.arange(-0.5, grid.shape[1], 1), minor=True)
    plt.yticks(np.arange(-0.5, grid.shape[0], 1), minor=True)

    # Mark Transmitter
    plt.scatter(tx[1], tx[0], color='green', s=100, label=f"Transmitter {tx}")

    # Mark all Receivers (free cells)
    rx_free = np.argwhere(grid == 0)
    plt.scatter(rx_free[:, 1], rx_free[:, 0], color='blue', s=40, label="Receivers (Free cells)")

    plt.legend()
    plt.title('Grid with Obstacles & All Signal Paths')
    plt.show()


In [5]:
import numpy as np
import matplotlib.pyplot as plt

def supercover_line(x0, y0, x1, y1):
    points = []
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    x, y = x0, y0
    n = 1 + dx + dy
    x_inc = 1 if x1 > x0 else -1
    y_inc = 1 if y1 > y0 else -1
    error = dx - dy
    dx *= 2
    dy *= 2

    for _ in range(n):
        points.append((x, y))

        if error > 0:
            x += x_inc
            error -= dy
        else:
            y += y_inc
            error += dx

    rows, cols = zip(*points)
    return np.array(rows), np.array(cols)

def coverage(grid,tx):
    covered = np.zeros_like(grid)
    # Loop over every cell in the grid
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]):
            if grid[i, j] == 0:  # Free cell: receiver
                rx = (i, j)
                # Get the path using supercover
                path_rows, path_cols = supercover_line(tx[0], tx[1], rx[0], rx[1])
                # Check if any obstacle is in the path
                obstacles = np.any(grid[path_rows, path_cols] == 1)
                if not obstacles:
                    # Mark the entire path as covered
                    covered[path_rows, path_cols] = 1
                else:
                    # Do not mark receiver if obstacle exists
                    covered[rx[0], rx[1]] = 0
    return covered


In [6]:
def count_blind_spots(grid, covered):
    # Blind spot: free in grid but not covered
    blind_spots = np.sum((grid == 0) & (covered == 0))
    return blind_spots

In [7]:
def brute_force_best_transmitter(grid):
    free_cells = np.argwhere(grid == 0)
    min_blind_spots = np.inf
    best_tx = []
    t=[]
    for tx in free_cells:
      t.append(tx)
      covered = coverage(grid, tx)
      blind_spots = count_blind_spots(grid, covered)

      if blind_spots<min_blind_spots:
          min_blind_spots=blind_spots
          best_tx=[tx]
      elif blind_spots == min_blind_spots:
            best_tx.append(tx)
    return min_blind_spots, best_tx


Grid with Obstacles:
[[0 0 0 0]
 [0 0 1 0]
 [0 0 0 0]
 [0 0 0 0]]


Blind Spots: 5
Best Transmitter Placement: [array([3, 2])]
Minimum Blind Spots: 1


In [8]:
A = 10  # Size of area
M = 16  # Number of cells

# Generate the grid
np.random.seed(42)#makes obstacle position constant that is obstacle at the same position every time
grid = create_grid(A, M, obstacle_prob=0.1)
print("Grid with Obstacles:")
print(grid)

tx=(0,1)
covered = coverage(grid,tx)
plot_grid(grid,tx)
# Count blind spots
blind_spots = count_blind_spots(grid, covered)
print(f"Blind Spots: {blind_spots}")

# Find best transmitter placement
min_blind_spots, best_tx = brute_force_best_transmitter(grid)
print(f"Best Transmitter Placement: {best_tx}")
print(f"Minimum Blind Spots: {min_blind_spots}")


# Classfication Task-II:All Obstacle Configuration


row  col  obstacle  tx_row  tx_column  covered  \
0    0    1         0       0          0        1   
1    2    0         1       0          0        0   
2    2    2         0       0          0        1   
3    1    2         0       0          0        1   
4    1    1         0       0          0        1   
5    0    2         1       0          0        0   
6    0    0         0       0          0        1   
7    2    1         0       0          0        1   
8    1    0         0       0          0        1   

   Blindspot(blocked=1/not blocked=0)  
0                                   0  
1                                   0  
2                                   0  
3                                   0  
4                                   0  
5                                   0  
6                                   0  
7                                   0  
8                                   0  


In [34]:
import pandas as pd
import matplotlib.pyplot as plt
def generate_dataset(grid_size, obstacle_prob=0.3,seen_configs=None):
    dataset_X = []
    dataset_y = []
    A = 10
    M = grid_size ** 2
    grid = create_grid(A, M, obstacle_prob)
    N = grid.shape[0]
    free_cells = [(i, j) for i in range(N) for j in range(N) if grid[i, j] == 0]
    if free_cells == []:
      return pd.DataFrame()
    tx=free_cells[0]
    plot_grid(grid,tx)
    covered = coverage(grid, tx)
    for i in range(N):
        for j in range(N):
            features = [
                i,
                j,
                grid[i, j],
                tx[0],
                tx[1],
                covered[i, j]
            ]
            if grid[i, j] == 0 and covered[i,j]==0:
                label = 1
            else:
              label = 0
            dataset_X.append(features)
            dataset_y.append(label)


    full_data = list(zip(dataset_X, dataset_y))
    np.random.shuffle(full_data)

    dataset_X, dataset_y = zip(*full_data)

    df = pd.DataFrame(dataset_X, columns=['row', 'col', 'obstacle', 'tx_row', 'tx_column', 'covered'])
    df['Blindspot(blocked=1/not blocked=0)'] = dataset_y

    return df

# Generate and print the dataset
train = generate_dataset(grid_size=3)
train.to_csv("train.csv", index=False)
print(train)

In [29]:
from itertools import product
import pandas as pd
import numpy as np

def generate_test_dataset_all_configs(grid_size):
    dataset_X = []
    dataset_y = []
    grid_ids = []
    seen_configs = set()

    total_cells = grid_size * grid_size
    all_configs = list(product([0, 1], repeat=total_cells))

    grid_id = 1

    for flat_config in all_configs:
        grid = np.array(flat_config).reshape((grid_size, grid_size))
        config_tuple = tuple(grid.flatten())

        if config_tuple in seen_configs:
            continue
        seen_configs.add(config_tuple)

        free_cells = [(i, j) for i in range(grid_size) for j in range(grid_size) if grid[i, j] == 0]

        if free_cells:
            tx = free_cells[0]
            covered = coverage(grid, tx)
        else:
            tx = (-1, -1)
            covered = np.zeros((grid_size, grid_size), dtype=int)

        for i in range(grid_size):
            for j in range(grid_size):
                features = [i, j, grid[i, j], tx[0], tx[1], covered[i, j]]
                label = 1 if grid[i, j] == 0 and covered[i, j] == 0 else 0
                dataset_X.append(features)
                dataset_y.append(label)
                grid_ids.append(grid_id)

        grid_id += 1

    df = pd.DataFrame(dataset_X, columns=['row', 'col', 'obstacle', 'tx_row', 'tx_column', 'covered'])
    df['Blindspot(blocked=1/not blocked=0)'] = dataset_y
    df['grid_id'] = grid_ids
    return df


In [35]:
test = generate_test_dataset_all_configs(grid_size=3)
test.to_csv("test.csv", index=False)
print(test.head())


   row  col  obstacle  tx_row  tx_column  covered  \
0    0    0         0       0          0        1   
1    0    1         0       0          0        1   
2    0    2         0       0          0        1   
3    1    0         0       0          0        1   
4    1    1         0       0          0        1   

   Blindspot(blocked=1/not blocked=0)  grid_id  
0                                   0        1  
1                                   0        1  
2                                   0        1  
3                                   0        1  
4                                   0        1  


In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
'''data = pd.read_csv("grid_dataset.csv")
data['tx_pos'] = '('+data['tx_row'].astype(str) + ',' + data['tx_column'].astype(str)+')'

unique_transmitters = data['tx_pos'].unique()
train_tx, test_tx = train_test_split(unique_transmitters, test_size=0.3, shuffle=True,random_state=30)
#print(train_tx.shape)
print(test_tx)'''

train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

X_train = train_data.drop(['Blindspot(blocked=1/not blocked=0)','covered'], axis=1)
y_train = train_data['Blindspot(blocked=1/not blocked=0)']
X_test = test_data.drop(['Blindspot(blocked=1/not blocked=0)', 'covered','grid_id'], axis=1)
y_test = test_data['Blindspot(blocked=1/not blocked=0)']

clf = RandomForestClassifier(n_estimators=100,bootstrap=True,random_state=30)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("\n## Blind Spot Classification Report\n")
print(classification_report(y_test, y_pred))



## Blind Spot Classification Report

              precision    recall  f1-score   support

           0       0.81      1.00      0.90      3751
           1       0.00      0.00      0.00       857

    accuracy                           0.81      4608
   macro avg       0.41      0.50      0.45      4608
weighted avg       0.66      0.81      0.73      4608



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
